In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from ipywidgets import interactive
import ipywidgets as widget

In [2]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter serverextension enable voila --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling: voila
- Writing config: c:\users\bgodf\onedrive\desktop\loan-fraud\env\etc\jupyter
    - Validating...
      voila 0.3.5 ok


In [3]:
#Converts the type from object into a category type.
def loan_categories(loans):
    for label, content in loans.items():
        if pd.api.types.is_string_dtype(content):
            loans[label] = content.astype('category').cat.as_ordered() 
    #This will change the data type from categorical into a number.
    for label, content in loans.items():
        if pd.api.types.is_categorical_dtype(content):
            loans[label] = pd.Categorical(content).codes
    return loans

In [4]:
model = pickle.load(open('Data/loan-fraud.pkl', 'rb'))

In [5]:
from IPython.display import display

#Creates a bar chart to show the probability of the transaction being fraud or not
def bar_chart(prob):
    probabilities = {'Not Fraud': prob[0],
                    'Fraud': prob[1]}
    fig, ax = plt.subplots()
    ax.set_title('Probability of Fraud')
    ax.set_ylabel('Percent Probability')
    ax.bar(probabilities.keys(), probabilities.values(), width=.5);


   

#Takes the values from the widgets and displays the prediction of fraud and the bar chart of the likelihood
def prediction_function(step, typeTransaction, amount, oldBalance, newBalance, oldBalanceDest, newBalanceDest):
    data = [[step, typeTransaction, amount, oldBalance, newBalance, oldBalanceDest, newBalanceDest]]
    features = pd.DataFrame(data, columns=['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 
                                           'oldbalanceDest', 'newbalanceDest'])
    features = loan_categories(features)
    yPreds = model.predict(features)
    probability = model.predict_proba(features)
    
    probability = probability * 100  
    bar_chart(probability[0])

    displayText = ''
    if yPreds[0] == 1:
        displayText = 'Fraud Likely'
    else:
        displayText = 'Fraud Unlikely'
    print(displayText)

In [6]:
dropdownOptions = ['CASH-IN', 'CASH-OUT', 'DEBIT', 'PAYMENT', 'TRANSFER']
style = {'description_width': 'initial'}
dis = interactive(prediction_function,
      step = widget.IntText(value=1,description='Steps in hours:', style=style),
      typeTransaction = widget.Dropdown(options=dropdownOptions, description='Type of Transaction', style=style), 
      amount = widget.FloatText(value = 0.0,description='Amount of loan:',style=style),
      oldBalance = widget.FloatText(value = 0.0,description='Original Balance Before:', style=style),
      newBalance = widget.FloatText(value = 0.0,description='Original Balance After:', style=style),
      oldBalanceDest = widget.FloatText(value = 0.0, description='Destination Balance Before:', style=style),
      newBalanceDest = widget.FloatText(value = 0.0,description='Destination Balance After:', style=style)
      )
display(dis)

interactive(children=(IntText(value=1, description='Steps in hours:', style=DescriptionStyle(description_width…

In [7]:
!pip freeze > requirements.txt